In [229]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import brentq
from sklearn.linear_model import SGDRegressor



### Virtual valuation and Reserve price calculator

In [221]:
class VirtualValuationCalculator:
    def __init__(self, stddev):
        self.stddev = stddev
    
    def virtual_valuation(self, v, mean):  # Calculate virtual valuation
        f_cdf = norm.cdf(v, loc=mean, scale=self.stddev)
        f_pdf = norm.pdf(v, loc=mean, scale=self.stddev)
        return v - (1 - f_cdf) / f_pdf


class ReservePriceCalculator:
    def __init__(self, means, stddev):
        self.means = means
        self.virtual_valuation_calculator = VirtualValuationCalculator(stddev)

    def phi_inverse_0(self, mean, stddev):
        """
        Use Brent's method to calculate the reserve price (phi^-1(0)).
        """
        def phi(v):
            return self.virtual_valuation_calculator.virtual_valuation(v, mean)

        # Use brentq for root-finding, requiring two initial guesses that bracket the root.
        v_low = mean - 3*stddev  # Lower bound for root-finding
        v_high = mean + 3*stddev  # Upper bound for root-finding
        reserve_price = brentq(phi, v_low, v_high)
        
        return reserve_price

    def calculate_reserve_prices(self,stddev):
        reserve_prices = []
        for mean in self.means:
            r_star = self.phi_inverse_0(mean, stddev)
            reserve_prices.append(r_star)
        return reserve_prices
    

### Set up participant data

In [222]:
# Example usage
np.random.seed(42)

# Simulating participants
num_participants = 10

true_means = np.random.uniform(50, 150, num_participants)  # True means of participants

initial_guess = 100 # Prior guess for every participant

est_means = np.full(num_participants, initial_guess) #array of these guesses which will be altered iteratively

# Store participant data
participants = {
    i: {
        "true_mean": true_means[i],
        "estimated_mean": est_means[i],
    }
    for i in range(num_participants)
}

In [223]:
stddev = 5 # Fixed standard deviation of 20 for simplicity of model

### Update particpant information to include true reserve pricing

In [224]:
calculator = ReservePriceCalculator(true_means, stddev)

# Calculate reserve prices for all participants
true_reserve_prices = calculator.calculate_reserve_prices(stddev)

# Update participant data with true reserve prices
for i, reserve_price in enumerate(true_reserve_prices):
    participants[i]["true_reserve_price"] = reserve_price

# Print the results
for i, data in participants.items():
    print(f"Participant {i+1}: {data}")

Participant 1: {'true_mean': 87.45401188473625, 'estimated_mean': 100, 'true_reserve_price': 77.82702917193446}
Participant 2: {'true_mean': 145.07143064099162, 'estimated_mean': 100, 'true_reserve_price': 134.15087322892364}
Participant 3: {'true_mean': 123.1993941811405, 'estimated_mean': 100, 'true_reserve_price': 112.67800408478666}
Participant 4: {'true_mean': 109.86584841970367, 'estimated_mean': 100, 'true_reserve_price': 99.6342731799071}
Participant 5: {'true_mean': 65.60186404424365, 'estimated_mean': 100, 'true_reserve_price': 56.79570493836255}
Participant 6: {'true_mean': 65.59945203362027, 'estimated_mean': 100, 'true_reserve_price': 56.7934025369315}
Participant 7: {'true_mean': 55.80836121681995, 'estimated_mean': 100, 'true_reserve_price': 47.496608820983724}
Participant 8: {'true_mean': 136.61761457749353, 'estimated_mean': 100, 'true_reserve_price': 125.84183888351781}
Participant 9: {'true_mean': 110.11150117432088, 'estimated_mean': 100, 'true_reserve_price': 99.87

### Include auctionner guess of reserve price per candidate

In [225]:
est_calculator = ReservePriceCalculator(est_means, stddev)

est_reserve_prices = est_calculator.calculate_reserve_prices(stddev)

for i, est_reserve in enumerate(est_reserve_prices):
    participants[i]['Estimated_reserve_price'] = est_reserve

for i, data in participants.items():
    print(f"Participant {i+1}: {data}")    

Participant 1: {'true_mean': 87.45401188473625, 'estimated_mean': 100, 'true_reserve_price': 77.82702917193446, 'Estimated_reserve_price': 90.01316311109564}
Participant 2: {'true_mean': 145.07143064099162, 'estimated_mean': 100, 'true_reserve_price': 134.15087322892364, 'Estimated_reserve_price': 90.01316311109564}
Participant 3: {'true_mean': 123.1993941811405, 'estimated_mean': 100, 'true_reserve_price': 112.67800408478666, 'Estimated_reserve_price': 90.01316311109564}
Participant 4: {'true_mean': 109.86584841970367, 'estimated_mean': 100, 'true_reserve_price': 99.6342731799071, 'Estimated_reserve_price': 90.01316311109564}
Participant 5: {'true_mean': 65.60186404424365, 'estimated_mean': 100, 'true_reserve_price': 56.79570493836255, 'Estimated_reserve_price': 90.01316311109564}
Participant 6: {'true_mean': 65.59945203362027, 'estimated_mean': 100, 'true_reserve_price': 56.7934025369315, 'Estimated_reserve_price': 90.01316311109564}
Participant 7: {'true_mean': 55.80836121681995, 'e

In [226]:
for i, data in participants.items():
    print(f"Particpant {i+1}: {data['true_reserve_price'],data['Estimated_reserve_price']}")

Particpant 1: (77.82702917193446, 90.01316311109564)
Particpant 2: (134.15087322892364, 90.01316311109564)
Particpant 3: (112.67800408478666, 90.01316311109564)
Particpant 4: (99.6342731799071, 90.01316311109564)
Particpant 5: (56.79570493836255, 90.01316311109564)
Particpant 6: (56.7934025369315, 90.01316311109564)
Particpant 7: (47.496608820983724, 90.01316311109564)
Particpant 8: (125.84183888351781, 90.01316311109564)
Particpant 9: (99.87419136028916, 90.01316311109564)
Particpant 10: (110.33486965151856, 90.01316311109564)


Eventually, we want to get the estimated reserve prices as close as possible to the true reserve pricings in order to optimsie our auction model.

### Bid function

In [227]:
def generate_bid(true_mean, stddev):
    return norm.rvs(true_mean, stddev)

### Payment using true reserve price

In [241]:
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import SGDRegressor

class Auction:
    def __init__(self, num_participants, stddev, true_means):
        self.num_participants = num_participants
        self.stddev = stddev
        self.true_means = true_means
        self.est_means = np.full(num_participants, 100)  # Initial estimates for each participant

        self.reserve_price_calculator = ReservePriceCalculator(true_means, stddev)

    def generate_bid(self, true_mean, stddev):
        """Generate a random bid based on true mean and standard deviation."""
        return norm.rvs(true_mean, stddev)

    def run_auction(self):
        """
        Run a single round of the auction.
        """
        # Generate bids for all participants
        bids = [self.generate_bid(mean, self.stddev) for mean in self.true_means]

        # Calculate reserve prices
        reserve_prices = self.reserve_price_calculator.calculate_reserve_prices(self.stddev)

        # Find the highest bid and second-highest bid
        sorted_bids = sorted(zip(bids, range(self.num_participants)), reverse=True)
        highest_bid, highest_idx = sorted_bids[0]
        second_highest_bid = sorted_bids[1][0]

        # Check if the highest virtual valuation exceeds the reserve price
        highest_virtual_valuation = self.reserve_price_calculator.virtual_valuation_calculator.virtual_valuation(
            highest_bid, self.true_means[highest_idx]
        )
        payment = None
        if highest_virtual_valuation >= reserve_prices[highest_idx]:
            # Item is allocated to highest bidder, apply payment rule
            payment = max(second_highest_bid, reserve_prices[highest_idx])

        # Update all participants' estimated means using their bids
        self.update_estimates(bids)

        # Collect results for this round
        round_results = {
            "bids": bids,
            "reserve_prices": reserve_prices,
            "highest_bid": highest_bid,
            "highest_idx": highest_idx,
            "second_highest_bid": second_highest_bid,
            "payment": payment,
            "updated_estimates": self.est_means.copy(),
        }

        # Print round results
        print(f"Round Results:")
        print(f"Bids: {bids}")
        print(f"Reserve Prices: {reserve_prices}")
        print(f"Highest Bid: {highest_bid} by Participant {highest_idx + 1}")
        print(f"Second Highest Bid: {second_highest_bid}")
        print(f"Payment: {payment}")
        print(f"Updated Estimates: {self.est_means}")
        print()

        return round_results

    def update_estimates(self, bids):
        """
        Use SGD to update the estimated mean for each participant based on their bids.
        """
        for idx, bid in enumerate(bids):
            # Use a simple learning rate (eta) for manual SGD updates
            learning_rate = 0.1
            self.est_means[idx] = (1 - learning_rate) * self.est_means[idx] + learning_rate * bid

# Run the auction simulation
np.random.seed(42)
num_participants = 10
true_means = np.random.uniform(50, 150, num_participants)  # True means of participants
stddev = 20  # Standard deviation

auction = Auction(num_participants, stddev, true_means)

# Simulate the auction for 10 rounds
all_rounds_data = []
for round_num in range(100):
    print(f"--- Round {round_num + 1} ---")
    round_data = auction.run_auction()
    all_rounds_data.append(round_data)





--- Round 1 ---
Round Results:
Bids: [78.0645241660372, 155.92263151271092, 113.93104032489126, 100.55125334829853, 70.44110947556433, 27.333847140464307, 21.310004566559293, 125.37186399267407, 89.8548787676324, 127.09220443151003]
Reserve Prices: [67.84040802697032, 117.62489730690612, 98.166387777819, 86.59362655849641, 50.79698635633805, 50.79519599227818, 43.73311128858694, 110.04401845823405, 86.80442492893235, 96.07171936409978]
Highest Bid: 155.92263151271092 by Participant 2
Second Highest Bid: 127.09220443151003
Payment: 127.09220443151003
Updated Estimates: [ 97 105 101 100  97  92  92 102  98 102]

--- Round 2 ---
Round Results:
Bids: [69.29353037431203, 116.8253566142858, 152.5123695595716, 105.35032240997296, 66.95242813800213, 37.10448830935113, 44.9207067263163, 138.83606637169086, 87.09162962587482, 128.321218146518]
Reserve Prices: [67.84040802697032, 117.62489730690612, 98.166387777819, 86.59362655849641, 50.79698635633805, 50.79519599227818, 43.73311128858694, 110.0